# EDOAL-2-CSV Tutorial

In [4]:
import xml.etree.ElementTree as ET
import pandas as pd
import re
import os
from utils import Parser

## Conference example

Load the EDOAL alignment you want to convert, extract its mappings and then save it as a CSV.

In [5]:
path = os.path.join("data", "cmt-conference-AMLC.edoal")
p = Parser(True)
df_AMLC = p.extract_mappings(path)
df_AMLC.to_csv(os.path.join("data","cmt-conference-AMLC.csv"), index=False)
df_AMLC.head()

NameError: name 'parse_entity' is not defined

In [ ]:
path = os.path.join("data", "cmt-conference-PGARM.edoal")
df_PGARM = u.extract_mappings(path)

In [ ]:
ref_path = os.path.join("data", "cmt-conference-REF.edoal")
df_eval1 = u.evaluate(df_PGARM, path, "full")
#df_eval1.to_csv(os.path.join("results","03_10","cmt_conference_diff.csv"), index=False)

In [ ]:
df_evaluated.merge(df_tobe, how='right').to_csv(os.path.join("results","04_13","unfiltered.csv"), index=False)

In [ ]:
# Filter out mappings bellow 0.5
df = format_cmt(df, "entity2")
df_filtered = df[(df.measure > 0.5)]
print("Filtered {} lines".format(df.shape[0]- df_filtered.shape[0]))

## Enslaved example

In [ ]:
path = os.path.join("data","enslaved.rdf")
df_enslaved = u.extract_mappings(path, False)
df_enslaved.head()

Entities from the second ontology (entity2) have IDs instead of proper names, so we can use a function to fetch more intelligeble labels from the ontology file.

In [ ]:
labels = u.get_labels(os.path.join("data","wikidataKG.owl"))
labels['https://lod.enslaved.org/entity/Q375']

In [ ]:
l = []
for i in ref_df['entity1']:
    l.append(i.replace('#','/'))

ref_['entity2'] = l

In [ ]:
ref_path = "enslaved-ref.edoal"
full_df = evaluate(df, ref_path, "full")
full_df.to_csv("enslaved_2101.csv", index=False)

In [ ]:
preview_df = evaluate(df, ref_path, "preview")
preview_df

In [ ]:

full_df = u.evaluate(df, ref_path, "missing")
full_df.to_csv("enslaved_missing.csv", index=False)

# Some statistics

### Mapping types

In [ ]:
# This function returns the number of different mappings types that exist in the given alignment
def check_mapping_types(df):
    return df.groupby(['ent1_type','ent2_type']).size().reset_index(name='Count')

In [ ]:
check_mapping_types(df)

### Simple vs complex mappings

In [ ]:
#Simple vs complex mappings
df['type_of_alignment'].value_counts()